I didn't have time to clean this notebook. Trying to not overuse free service pushed me to some unnecesary repeating after cleaning iteratitions.

Basicaly it is very simple:
- load custormer and libraries addreses
- filter out duplicates
- query the service
- save resultes in csv file as cache
- use csv file in other notebook

In [38]:
from geopy.geocoders import Nominatim
import time
from shapely import Point
import pandas as pd
import geopandas as gpd
import datetime 

geolocator = Nominatim(user_agent="marko@droopia.net")

def get_gps_coordinates(address):
    """
    Transform address to GPS coordinates.

    Example usage
    get_gps_coordinates('6008 SE 49TH AVE, PORTLAND, OR')
    """
    time.sleep(1.4)
    location = geolocator.geocode(address)
    if location:
        return location.longitude, location.latitude
    else:
        return None


def split_point(point_str):
    """
    # Remove the "POINT (" and ")" part from the string
    # Example usage
    point = "POINT (-122.6187 45.5234)"
    longitude, latitude = split_point(point)
    """

    point_str = point_str.replace("POINT (", "").replace(")", "")
    longitude, latitude = point_str.split()

    return longitude, latitude



In [5]:
customers_df = pd.read_pickle('data/customers_df.pickle')

In [6]:
libraries_df = pd.read_pickle('data/libraries_df.pickle')

In [9]:
customers_df.head(2)

,patron_id,customer_name,customer_street_address,customer_city,customer_state,customer_zipcode,customer_birth_date,customer_gender,customer_education,customer_occupation,customer_years_old,customer_address,geometry,longitude,latitude
0,df83ec2d0d409395c0d8c2690cfa8b67,Cynthia Barnfield,44 NE MEIKLE PL,PORTLAND,OREGON,97213,2009-09-10,FEMALE,HIGHSCHOOL,NA,11,"44 NE MEIKLE PL, PORTLAND, OREGON",POINT (-122.61875 45.52349),-122.61875283330345,45.52348805
1,6aec7ab2ea0d67161dac39e5dcabd857,Elizabeth Smith,7511 SE HARRISON ST,PORTLAND,OREGON,97215,1956-12-15,FEMALE,COLLEGE,BLUECOLLAR,64,"7511 SE HARRISON ST, PORTLAND, OREGON",POINT (-122.58601 45.50912),-122.58601213138911,45.509121


In [10]:
address_list = customers_df['customer_address'].drop_duplicates().to_list() + libraries_df['address'].drop_duplicates().to_list()

Load cached coordinates:

In [11]:
coordinates_list_df = pd.read_pickle('data/coordinates_list_gdf.pickle')

In [252]:
# coordinates_list_df = gpd.GeoDataFrame()
# coordinates_list_df = pd.concat([coordinates_list_df, pd.read_csv('data/coordinates_list_1293.csv').drop('Unnamed: 0', axis=1)])
# coordinates_list_df = pd.concat([coordinates_list_df, pd.read_csv('data/coordinates_list_1915.csv').drop('Unnamed: 0', axis=1)])
# coordinates_list_df = pd.concat([coordinates_list_df, pd.read_csv('data/coordinates_list_3.csv').drop('Unnamed: 0', axis=1)])
# coordinates_list_df = pd.concat([coordinates_list_df, pd.read_csv('data/coordinates_list_4.csv').drop('Unnamed: 0', axis=1)])
# coordinates_list_df = pd.concat([coordinates_list_df, pd.read_csv('data/coordinates_list_NEW5.csv').drop('Unnamed: 0', axis=1)])

# # pd.read_csv('data/coordinates_list_1915.csv').drop('Unnamed: 0', axis=1)
# coordinates_list_df = coordinates_list_df.drop('Unnamed: 3', axis=1)

In [12]:
# coordinates_list_df.to_csv('data/coordinates_list.csv')

In [13]:
coordinates_list_df.head(2)

,address,geometry,longitude,latitude
0,"44 NE MEIKLE PL, PORTLAND, OREGON",POINT (-122.61875 45.52349),-122.61875283330345,45.52348805
1,"7511 SE HARRISON ST, PORTLAND, OREGON",POINT (-122.58601 45.50912),-122.58601213138911,45.509121


In [2]:
# coordinates_list_df.head(5)

In [151]:
# coordinates_list_gdf[coordinates_list_gdf.address.str.contains('N/A')]

In [14]:
coordinates_list_df[coordinates_list_df.geometry.isna()]

,address,geometry,longitude,latitude


In [15]:
coordinates_list_df = coordinates_list_df[~coordinates_list_df.geometry.isna()].copy()

In [18]:
coordinates_list_df[['longitude', 'latitude']] = coordinates_list_df.geometry.apply(lambda x: pd.Series(split_point(str(x))))

In [19]:
coordinates_list_df.head(2)

,address,geometry,longitude,latitude
0,"44 NE MEIKLE PL, PORTLAND, OREGON",POINT (-122.61875 45.52349),-122.61875283330345,45.52348805
1,"7511 SE HARRISON ST, PORTLAND, OREGON",POINT (-122.58601 45.50912),-122.58601213138911,45.509121


In [20]:
coordinates_list_df['longitude'] = coordinates_list_df.apply(lambda x: x.longitude.replace('(','').replace(',',''), axis=1) 

In [21]:
coordinates_list_df['geometry'] = coordinates_list_df.apply(lambda x: Point(x.longitude.replace('(','').replace(',',''), x.latitude) , axis=1) 

In [22]:
coordinates_list_df.head(2)

,address,geometry,longitude,latitude
0,"44 NE MEIKLE PL, PORTLAND, OREGON",POINT (-122.61875 45.52349),-122.61875283330345,45.52348805
1,"7511 SE HARRISON ST, PORTLAND, OREGON",POINT (-122.58601 45.50912),-122.58601213138911,45.509121


In [23]:
coordinates_list_gdf = gpd.GeoDataFrame(coordinates_list_df, geometry='geometry', crs='EPSG:4326')

In [24]:
coordinates_list_gdf.head(2)

,address,geometry,longitude,latitude
0,"44 NE MEIKLE PL, PORTLAND, OREGON",POINT (-122.61875 45.52349),-122.61875283330345,45.52348805
1,"7511 SE HARRISON ST, PORTLAND, OREGON",POINT (-122.58601 45.50912),-122.58601213138911,45.509121


In [25]:
coordinates_list_gdf = coordinates_list_gdf[~coordinates_list_gdf.address.str.contains('N/A')]

Test service and our data

In [27]:
get_gps_coordinates('6008 SE 49TH AVE, PORTLAND, OR')

(-122.61210650846269, 45.47900505)

In [28]:
coordinates_list_df[coordinates_list_df.address=='6008 SE 49TH AVE, PORTLAND, OR']

,address,geometry,longitude,latitude
5,"6008 SE 49TH AVE, PORTLAND, OR",POINT (-122.61211 45.47901),-122.61210650846269,45.47900505


In [29]:
# address_list_2 = []
# for a in address_list:
#     if ('N/A' in a):
#         print(a)
#     else: 
#         address_list_2.append(a)

In [30]:
# len(address_list_2)

# address_list_3 = []
# for a in address_list_2:
#     if ('NA' in a):
#         print(a)
#     # else: 
#     #     address_list_3.append(a)

In [31]:
# coordinates_list_df

In [32]:
# address_list_3 = pd.read_csv('data/customer_new5.csv').customer_address.to_list()

Query only coordinates we don't have in our cache

In [267]:
coordinates_list = []
i = 0

for address in address_list:
    i = i + 1
    if (coordinates_list_df[coordinates_list_df.address==address].shape[0] == 0):
        print(i, address)
        coordinates_list.append({
            'address': address,
            'geometry': get_gps_coordinates(address)
        })
        # print(i, address)
    gpd.GeoDataFrame(coordinates_list).to_csv('data/coordinates_list_NEW5.csv')


In [250]:
gpd.GeoDataFrame(coordinates_list)

,address,geometry
0,"4960 SW 31ST DR, PORTLAND, OREGON","(-122.709383, 45.486966249999995)"
1,"14230 SW ALPINE CREST WAY, PORTLAND, OREGON",None
2,"3624 N LEVERMAN ST, PORTLAND, OREGON","(-122.7051396122449, 45.569292469387754)"
3,"1920 NW 18TH AVE, PORTLAND, OREGON","(-122.68961548979591, 45.53659271428571)"
4,"1201 SE GIDEON ST, PORTLAND, OREGON","(-122.6540210909091, 45.50285397979798)"
...,...,...
194,"10477 SE WAVERLY CT, PORTLAND, OREGON",None
195,"5516 NE ST JAMES RD, VANCOUVER, WASHINGTON","(-122.63855134857835, 45.662288000000004)"
196,"7011 SE NORBERT DR, PORTLAND, OREGON",None
197,"6025 NE PRESCOTT ST, PORTLAND, OREGON","(-122.60052433333333, 45.55553112121212)"


In [225]:
coordinates_list_df = gpd.GeoDataFrame(coordinates_list)

In [ ]:
coordinates_list_gdf = gpd.GeoDataFrame(coordinates_list_df, geometry='geometry', crs='EPSG:4326')

coordinates_list_df.to_pickle('data/coordinates_list_df.pickle')
coordinates_list_gdf.to_pickle('data/coordinates_list_gdf.pickle')